In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import anndata
import scanpy as sc
from ALLCools.mcds import MCDS
from ALLCools.clustering import tsne, significant_pc_test, filter_regions, remove_black_list_region, lsi, binarize_matrix
from ALLCools.plot import *
from ALLCools.clustering import ConsensusClustering, Dendrogram, get_pc_centers

4 6  
S1   010101   
S2   110011  
S3   0--100  
S4   10--10  

In [2]:
metadata = pd.read_csv('Chaligne2021NG/chaligne_metadata.csv', index_col = 0)
mcds = MCDS.open('Chaligne2021NG/Chaligne2021NG/Chaligne2021NG.mcds/', var_dim='chrom5k')

In [3]:
mcds = MCDS.open('Chaligne2021NG/Chaligne2021NG/Chaligne2021NG.mcds/', var_dim='chrom5k')
mcds.add_cell_metadata(metadata)
selc = metadata.index
mcds = mcds.sel(cell = mcds.get_index('cell').intersection(selc))
mcds

/gale/netapp/home2/yuwu/miniconda3/envs/allcools/lib/python3.8/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.MCDS>
Dimensions:                    (cell: 1578, chrom5k: 617669)
Coordinates:
  * cell                       (cell) object 'GBM_MGH105C_A1_S1_L008.ACAACC' ...
  * chrom5k                    (chrom5k) <U11 'chr1_0' 'chr1_1' ... 'chrY_11445'
    chrom5k_chrom              (chrom5k) <U5 dask.array<chunksize=(617669,), meta=np.ndarray>
    chrom5k_end                (chrom5k) int64 dask.array<chunksize=(617669,), meta=np.ndarray>
    chrom5k_start              (chrom5k) int64 dask.array<chunksize=(617669,), meta=np.ndarray>
    cell_# reads (x10^6)       (cell) float64 2.712 3.512 3.127 ... 3.564 3.325
    cell_# CpGs (x10^6)        (cell) float64 0.1385 0.186 ... 0.1358 0.2973
    cell_CG conversion rate    (cell) float64 99.8 99.8 99.8 ... 99.8 99.8 99.8
    cell_Sample                (cell) object 'MGH105C' 'MGH105C' ... 'MGH211'
    cell_Type                  (cell) object 'GBM' 'GBM' 'GBM' ... 'GBM' 'GBM'
Data variables:
    chrom5k_da_CGN-hypo-score  (cell, chrom5k) float16 dask.array<chunksize=(1578, 42393), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom5k

In [4]:
mcds = mcds.remove_black_list_region(black_list_path='/gale/netapp/home/hanliu/ref/blacklist/hg38-blacklist.v2.bed.gz')
chrom_to_remove = ['chrL', 'chrM', 'chrX', 'chrY']
mcds = mcds.remove_chromosome( chrom_to_remove,'chrom5k')
#mcds.add_mc_frac(var_dim='chrom5k', normalize_per_cell=True, clip_norm_value=10)
mcds

45816 chrom5k features removed due to overlapping (bedtools intersect -f 0.2) with black list regions.
34491 chrom5k features in ['chrL', 'chrM', 'chrX', 'chrY'] removed.


<xarray.MCDS>
Dimensions:                    (cell: 1578, chrom5k: 537362)
Coordinates:
  * cell                       (cell) object 'GBM_MGH105C_A1_S1_L008.ACAACC' ...
  * chrom5k                    (chrom5k) <U11 'chr1_159' ... 'chr9_27643'
    chrom5k_chrom              (chrom5k) <U5 dask.array<chunksize=(537362,), meta=np.ndarray>
    chrom5k_end                (chrom5k) int64 dask.array<chunksize=(537362,), meta=np.ndarray>
    chrom5k_start              (chrom5k) int64 dask.array<chunksize=(537362,), meta=np.ndarray>
    cell_# reads (x10^6)       (cell) float64 2.712 3.512 3.127 ... 3.564 3.325
    cell_# CpGs (x10^6)        (cell) float64 0.1385 0.186 ... 0.1358 0.2973
    cell_CG conversion rate    (cell) float64 99.8 99.8 99.8 ... 99.8 99.8 99.8
    cell_Sample                (cell) object 'MGH105C' 'MGH105C' ... 'MGH211'
    cell_Type                  (cell) object 'GBM' 'GBM' 'GBM' ... 'GBM' 'GBM'
Data variables:
    chrom5k_da_CGN-hypo-score  (cell, chrom5k) float16 dask.array<chunksize=(1578, 37829), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom5k

In [6]:
df = mcds['chrom5k_da_CGN-hypo-score'][0:100,0:100].to_pandas()
df

chrom5k,chr1_159,chr1_160,chr1_161,chr1_162,chr1_163,chr1_164,chr1_165,chr1_166,chr1_167,chr1_168,...,chr1_249,chr1_250,chr1_251,chr1_252,chr1_253,chr1_254,chr1_255,chr1_256,chr1_257,chr1_258
cell,,,,,,,,,,,,,,,,,,,,,
GBM_MGH105C_A1_S1_L008.ACAACC,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
GBM_MGH105C_A1_S1_L008.ACAGAC,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
GBM_MGH105C_A1_S1_L008.ACTCAC,0.0,0.0,0.0,0.0,0.0,0.0,0.986816,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0
GBM_MGH105C_A1_S1_L008.AGAAGG,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0
GBM_MGH105C_A1_S1_L008.AGGATG,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.967285,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GBM_MGH121_Plate1_C1_S2_L006.CAAGAG,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0
GBM_MGH121_Plate1_C1_S2_L006.CATGAC,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0
GBM_MGH121_Plate1_C1_S2_L006.CCATAG,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0


In [19]:
df = df.round().astype(str)

In [20]:
''.join(df.loc['GBM_MGH105C_A1_S1_L008.ACAACC',:].values)

'0000000000000000000000000000000000000000010010000000010000000000000000000000000000000000000000000000'

In [24]:
with open('test.phy', 'w') as f:
    f.write(str(len(df.index)) + ' ' + str(len(df.iloc[0,:])) + '\n')
    for i in df.index:
        f.write(i+'\t')
        f.write(''.join(df.loc[i,:].values))
        f.write('\n')
    f.close()

In [36]:
import toytree

newick = ''
with open('../../test_GBM.treefile', 'r') as f:
    newick = f.read()

tree = toytree.tree(newick)
canvas, axes, mark = tree.draw(use_edge_lengths=False)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="425.0px" height="1000.0px" viewBox="0 0 425.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t98237c0ce219454cb14e956f18286b6f"> GBM_MGH105C_G1_S4_L008.CATGAC GBM_MGH105C_A1_S1_L008.AGAAGG GBM_MGH105C_A1_S1_L008.CCATAG GBM_MGH105C_A1_S1_L008.ACTCAC GBM_MGH121_Plate1_C1_S2_L006.ACAGAC GBM_MGH121_Plate1_A1_S1_L006.ACTCAC GBM_MGH105C_E1_S3_L008.GCATTC GBM_MGH105C_G1_S4_L008.TGACAG GBM_MGH105C_E1_S3_L008.ACTCAC GBM_MGH121_Plate1_A1_S1_L006.GTTGAG GBM_MGH121_Plate1_A1_S1_L006.CTATTG GBM_MGH105C_C1_S2_L008.TGTAGG GBM_MGH105C_G1_S4_L008.AGAAGG GBM_MGH121_Plate1_A1_S1_L006.AGAAGG GBM_MGH105C_G1_S4_L008.CGGTAG GBM_MGH105C_E1_S3_L008.AGAAGG GBM_MGH105C_E1_S3_L008.CATGAC GBM_MGH105C_E1_S3_L008.ACAGAC GBM_MGH105C_C1_S2_L008.GCATTC GBM_MGH105C_C1_S2_L008.CGGTAG GBM_MGH121_Plate1_C1_S2_L006.CGGTAG GBM_MGH121_Plate1_A1_S1_L006.TGTAGG GBM_MGH121_Plate1_A1_S1_L006.CATGAC GBM_MGH105C_G1_S4_L008.GAAGTC GBM_MGH121_Plate1_A1_S1_L006.TCTCTG GBM_MGH121_Plate1_A1_S1_L006.GGTAAC GBM_MGH105C_C1_S2_L008.AGGATG GBM_MGH105C_E1_S3_L008.TGACAG GBM_MGH105C_C1_S2_L008.GGTAAC GBM_MGH105C_G1_S4_L008.ACAGAC GBM_MGH121_Plate1_A1_S1_L006.GCATTC GBM_MGH121_Plate1_A1_S1_L006.CCATAG GBM_MGH121_Plate1_C1_S2_L006.ACAACC GBM_MGH121_Plate1_A1_S1_L006.AGGATG GBM_MGH121_Plate1_A1_S1_L006.GAAGTC GBM_MGH105C_E1_S3_L008.ATCGAC GBM_MGH105C_C1_S2_L008.TCTCTG GBM_MGH105C_G1_S4_L008.ACAACC GBM_MGH105C_E1_S3_L008.CTCAGC GBM_MGH105C_A1_S1_L008.ATCGAC GBM_MGH105C_E1_S3_L008.TGCTGC GBM_MGH105C_G1_S4_L008.CCATAG GBM_MGH105C_A1_S1_L008.GTGAGG GBM_MGH105C_C1_S2_L008.AGAAGG GBM_MGH105C_A1_S1_L008.CTATTG GBM_MGH121_Plate1_A1_S1_L006.ATCGAC GBM_MGH105C_A1_S1_L008.AGGATG GBM_MGH105C_C1_S2_L008.CCTTCG GBM_MGH105C_A1_S1_L008.GGTAAC GBM_MGH105C_A1_S1_L008.CAAGAG GBM_MGH121_Plate1_C1_S2_L006.AGGATG GBM_MGH105C_A1_S1_L008.TGCTGC GBM_MGH105C_G1_S4_L008.TGCTGC GBM_MGH105C_C1_S2_L008.ATCAAG GBM_MGH121_Plate1_A1_S1_L006.TATCTC GBM_MGH121_Plate1_C1_S2_L006.CATGAC GBM_MGH121_Plate1_C1_S2_L006.ACTCAC GBM_MGH121_Plate1_C1_S2_L006.CAAGAG GBM_MGH105C_G1_S4_L008.TCTCTG GBM_MGH105C_G1_S4_L008.GGTAAC GBM_MGH121_Plate1_A1_S1_L006.TGCTGC GBM_MGH121_Plate1_A1_S1_L006.ACAGAC GBM_MGH105C_G1_S4_L008.ACTCAC GBM_MGH105C_A1_S1_L008.TGACAG GBM_MGH105C_C1_S2_L008.ATCGAC GBM_MGH121_Plate1_C1_S2_L006.CCATAG GBM_MGH121_Plate1_A1_S1_L006.CTCAGC GBM_MGH105C_C1_S2_L008.ACAACC GBM_MGH105C_G1_S4_L008.TATCTC GBM_MGH105C_C1_S2_L008.ACAGAC GBM_MGH105C_E1_S3_L008.ATCAAG GBM_MGH121_Plate1_A1_S1_L006.CCTTCG GBM_MGH105C_A1_S1_L008.CGGTAG GBM_MGH105C_G1_S4_L008.GTGAGG GBM_MGH121_Plate1_A1_S1_L006.GTGAGG GBM_MGH121_Plate1_A1_S1_L006.CAAGAG GBM_MGH121_Plate1_A1_S1_L006.TGACAG GBM_MGH105C_C1_S2_L008.CATGAC GBM_MGH105C_A1_S1_L008.CATGAC GBM_MGH105C_A1_S1_L008.TGTAGG GBM_MGH105C_E1_S3_L008.TATCTC GBM_MGH121_Plate1_C1_S2_L006.ATCGAC GBM_MGH105C_C1_S2_L008.GTGAGG GBM_MGH105C_G1_S4_L008.TGTAGG GBM_MGH105C_C1_S2_L008.GAAGTC GBM_MGH105C_C1_S2_L008.CCATAG GBM_MGH105C_E1_S3_L008.CCTTCG GBM_MGH105C_E1_S3_L008.GAAGTC GBM_MGH105C_A1_S1_L008.GTTGAG GBM_MGH105C_A1_S1_L008.GAAGTC GBM_MGH105C_A1_S1_L008.TCTCTG GBM_MGH105C_G1_S4_L008.GCATTC GBM_MGH121_Plate1_C1_S2_L006.ATCAAG GBM_MGH105C_C1_S2_L008.TATCTC GBM_MGH105C_A1_S1_L008.CCTTCG GBM_MGH105C_C1_S2_L008.CTCAGC GBM_MGH121_Plate1_C1_S2_L006.CCTTCG GBM_MGH105C_A1_S1_L008.ACAGAC GBM_MGH105C_C1_S2_L008.GTTGAG GBM_MGH105C_A1_S1_L008.ACAACC